In [1]:
# Classics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

In [2]:
chorus = pd.read_csv('./chorus.csv')
verse = pd.read_csv('./verse.csv')

##  Verse Training

In [5]:
verse_text = ' '.join(verse['lyric'])

In [6]:
chars = sorted(list(set(verse_text)))

char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
maxlen = 40
steps = 10

sentences = []
next_chars = []

for i in range(0, len(verse_text) - maxlen, steps):
    sentences.append(verse_text[i: i + maxlen])
    next_chars.append(verse_text[i + maxlen])
    
print('There are', len(sentences), 'sentences')

There are 8896 sentences


In [8]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_int[char]] = 1
        
    y[i, char_to_int[next_chars[i]]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape = (maxlen, len(chars))))
model.add(Dense(len(chars), activation = 'relu'))
model.add(Dense(len(chars), activation = 'relu'))
model.add(Dense(len(chars), activation = 'relu'))
model.add(Dense(len(chars), activation = 'softmax'))

optimizer = RMSprop()
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

In [10]:
import sys

def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(verse_text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)
        
        generated = ''
        sentence = verse_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(60):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_int[char]] = 1
                
            preds = model.predict(x_pred, verbose = 0)[0]
            next_index = sample(preds, diversity)
            next_char = int_to_char[next_index]
            
            sentence = sentence[1:] + next_char
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)

In [ ]:
# model.fit(x, y,
#          batch_size = 128,
#          epochs = 10000,
#          callbacks = [print_callback])